# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv


# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cannonvale,-20.2833,148.7000,19.66,80,44,5.22,AU,1691615540
1,1,hermanus,-34.4187,19.2345,13.00,85,0,4.36,ZA,1691615540
2,2,port-aux-francais,-49.3500,70.2167,2.65,78,92,12.29,TF,1691615540
3,3,waitangi,-43.9535,-176.5597,9.90,77,78,2.24,NZ,1691615540
4,4,iquique,-20.2208,-70.1431,19.02,74,12,4.12,CL,1691615498


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
#%%capture --no-display

# Configure the map plot

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"  

)

# Display the map

map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
#Narrow down cities that fit criteria and drop any results with null values


filtered_weather1_df = city_data_df.loc[(city_data_df["Max Temp"] >= 15) & (city_data_df["Max Temp"] <= 25)]
##Cloudiness Filter

filtered_weather2_df = filtered_weather1_df.loc[(filtered_weather1_df["Humidity"] >= 20) & (filtered_weather1_df["Humidity"] <= 60)]
# #Wind Speed filter

filtered_weather3_df = filtered_weather2_df.loc[(filtered_weather2_df["Wind Speed"] <= 5)]
# #Drop any rows with null values

filtered_weather_Final_df = filtered_weather3_df.dropna()



# Display sample data
filtered_weather_Final_df.head()





,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,tiksi,71.6872,128.8694,16.46,52,8,3.17,RU,1691615549
38,38,thompson,55.7435,-97.8558,20.93,33,61,3.70,CA,1691615531
45,45,whitehorse,60.7161,-135.0538,20.01,47,48,1.34,CA,1691615490
56,56,yorkton,51.2167,-102.4677,22.03,32,100,3.66,CA,1691615562
58,58,klyuchi,52.2667,79.1667,19.64,46,80,3.08,RU,1691615563


In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel1_df = pd.DataFrame(filtered_weather_Final_df, columns=["City", "Country", "Lat", "Lng","Humidity"])


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel1_df["Hotel Name"] = ""

# Display sample data
hotel_df = hotel1_df.reset_index(drop=True)

hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,tiksi,RU,71.6872,128.8694,52,
1,thompson,CA,55.7435,-97.8558,33,
2,whitehorse,CA,60.7161,-135.0538,47,
3,yorkton,CA,51.2167,-102.4677,32,
4,klyuchi,RU,52.2667,79.1667,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"

params = {
"categories":categories,
"limit": limit,
"apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary

    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):

        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tiksi - nearest hotel: Арктика
thompson - nearest hotel: Thompson Inn
whitehorse - nearest hotel: Town & Mountain Hotel
yorkton - nearest hotel: Ramada Yorkton
klyuchi - nearest hotel: No hotel found
smithers - nearest hotel: Sunshine Inn Hotel
mumbwa - nearest hotel: La Hacienda
celendin - nearest hotel: Hotel Venecia
kemin - nearest hotel: Кемин
varva - nearest hotel: No hotel found
makinsk - nearest hotel: Береке
khovd - nearest hotel: Khovd hotel
ust-nera - nearest hotel: гостиница "Солнечная"
karratha - nearest hotel: Karratha International Hotel
south hedland - nearest hotel: Port Haven
bayanaul - nearest hotel: Жан
balqash - nearest hotel: Престиж
bonner - nearest hotel: No hotel found
oostduinkerke - nearest hotel: Hotel Hof Ter Duinen
catabola - nearest hotel: No hotel found
boysun - nearest hotel: Grand Hotel
caconda - nearest hotel: No hotel found
salima - nearest hotel: Palms lodge
kyzyl-suu - nearest hotel: кафе суусая
la ronge - nearest hotel: La Ron

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tiksi,RU,71.6872,128.8694,52,Арктика
1,thompson,CA,55.7435,-97.8558,33,Thompson Inn
2,whitehorse,CA,60.7161,-135.0538,47,Town & Mountain Hotel
3,yorkton,CA,51.2167,-102.4677,32,Ramada Yorkton
4,klyuchi,RU,52.2667,79.1667,46,No hotel found
5,smithers,CA,54.7804,-127.1743,41,Sunshine Inn Hotel
6,mumbwa,ZM,-14.9783,27.0619,35,La Hacienda
7,celendin,PE,-6.8703,-78.1517,45,Hotel Venecia
8,kemin,KG,42.7846,75.6892,44,Кемин
9,varva,UA,50.4950,32.7197,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
#%%capture --no-display

# Configure the map plot


hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols = ["Hotel Name", "Country"],
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"

)


# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)